# July 10

### Assembly

An assembly is a heirarchical data structure that maps the sequence data to a putative reconstruction of the target

* Only way to assemble chromosomes or relate your assembly back to specific chromosomes is to do genetic crossing
    * Generally, we will be talking about scaffold-level sequencing
    
Contigs: contiguous, ungapped seuqnce
* Groups reads together
* provides a multiple sequence alignment
* Provides a consensus sequence

Scaffolds: supercontigs
* Defines contig order and orientation
* defines size of gaps between contigs

### Sequence reads
* Sanger sequencing: 600-800bp
    * Use fluorescent, chain terminating nucleotides bind to a DNA strand, then you can run a gel and check the sequence as it's being built up
    * High quality reads -> low error rate


* Second generation
    * Roche 454
        * Same kind of concept as Sanger, but uses beads to amplify DNA, then reads from beads
    * Early Illumina (2007-2008)
        * 36bp reads orginally, high error rate, but could get 10M reads in one lane
    * Illumina
        * 100bp, 150bp, 250bp depending on the machine, good quality reads
        * 400M reads per lane
        * uses flow cells, clonal bridge amplification, then elongate with fluorescent primers and read colors
    
    * Illumina reads
        * Insert length of ~600bp
        * Paired end sequencing reads 100bp from both sides
        * allows you to possibly find gaps between scaffolds if you need to
        
        * Mate-pair reads
            * 2-10k DNA molecule (2kb, 5kb, 10kb), circularized with Biotin
            * Sonicate,then sequence the junction. Can use it to find larger gaps
            * Used to arrange contigs
            * 40kb mate-pairs by circularizing and shearing fosmids
            * 150-200kb mate-pairs by circularizing and shearing BACs
            
            
* Third generation
    * Synthetic single molecule sequencing
        * Moleculo - 10kb median length
        * 10x Genomics - sparse, full molecule sequencing
            * Shear long pieces of DNA and suspend in oil droplets, each droplet gets its own barcode, samples from the same initial piece of DNA should have barcodes close in number 
    
    * True single molecule sequencing
        * PacBio - 15kb median length, up to 25kb-50kb (read lengths are bound by length of initial DNA)
            * Error rate ~ 15%, truly random error rate (vs sequence specific)
        * Nanopore - 100kb???
            * error rate >> 15%
            * maybe can be used for final scaffolding since reads are so long


* Assembly Concepts
    * Overlap / Layout / Consensus
        * At it's most basic, just overlay each gene fragment and find the matching sequences, where the sequences are the nodes and links are the edges
        * **Hamiltonian cycle: visit each vertex one time**
        
        1. Overlaps must be discovered by an all-versus-all read comparison
            * Seed and extend algorithm is used: k-mer matching followed by alignment extension
            * Big O - O(n^2) runtime. Have to compare everything to everything else, runtime increases exponentially
        2. Contruct the overlap graph
            * Reads are nodes, overlaps are edges
            * Must deal with cycles and tangles in the graph
        3. The precise layout and consensus sequence is determined by doing a multiple sequence alignment
            * Progressive pair-wise alignments
            
        Newbler:
            * Construct unitigs: uncontested subgraphs
            * Seed a second overlap graph with unitigs to construct contigs
        
        * Because every read is a node, the number of nodes scales with the number of reads
        * Overlap computations become intractable on millions of reads
        * BUT, if read lengths increase substntially, OLC may once again be preferable
            * With PacBio, it is
            
    * Sequencing by Hybridization
        * A sequencing chip containing all possible, say, 3-mers (4^3 = 64 3-mers). Wash DNA across chip, DNA will hybridize to 3-mers contained within its sequence. Look at fluorescence to assemble 3-mers back into full sequence
        
        * K-mers, a string with length of k
            * Numbers of k-mers in a string = strlen - k + 1
            * Number of possible k-mers: 4^k. As k approaches 19, approaches unique sequences
            
     * De Bruijn graphs
         * Instead of each read representing a node in the graph, we will now let each unique sequence in the genome be represented by a node.
         * We can represent unique sequence with k-mers
         * Vertices (nodes) are (k-1)-mers, edges are k-mers
         * **A Eulerian path: visit each edge once**
         
         * Assembly is a byproduct of graph construction
         * Construct k-mers from reads, each read induces a path
         * K-mer hashing finds all k-mer overlaps
             * Ask if you've seen a k-mer before, if you haven't start a path, if you have, go to that path and continue
             
         * If:
             1. All k-mers are present in the genome
             2. All k-mers are error free
             3. Each k-mer appears exactly once
             (None of these will be true)
         * Then, a Eulerian path can be found and the genome can be perfectly reconstructed from the path
         
         
* Sequencing error
    * One sequencing error induces up to K errorneous k-mers
    * "Spurs" or "tips" are introduced (in De Bruijn) by sequencing errors, assemblers will find them and cut them out
    * Bubbles are induced by sequencing error or polymorphism
    * Repeat sequences cause path convergence (a repeat with 41 k-mers will give same 41 k-mers each time, they will be collapsed down
    * Cycles are introduced by short repeats
    
    * Bandage for De Bruijn graph visualization
    
  
* Sequencing Strategies
    * Not really proper to just sequence then find an appropriate assembly method after the fact
    * Algorithms tuned to assembly certain types of sequencing
    
    1. ALLPATHS-LG
        * 2x100bp Illumina reads
            * 180bp insert length (Overlapping ends)
            * 800bp insert length
            * 2kb, 5kb, 10kb mate pairs
            * 20x PacBio to try to join scaffolds
     
     2. Discovar *de novo* Strategy
         * 2x250bp Illumina Reads
             * 800bp insert length
             * 2kb, 5kb, 10kb mate pairs
             * 20kb PacBio
         * JGI's Meraculous assembler has been adapted to this
         
     3. BGI Strategy
         * SOAP *de novo*
         * Many PE insert lengths
         
     4. 10X Strategy
         * 1 10X Illumina 2x150bp
         
     5. Pure PacBio Strategy
         * 60x PacBio coverage

# July 11
### Short-read alignment

* Gene expression quantification
    * reads align to exons, have to use annotation file to assemble those back to the exons to check expression
    * Aligners have to make sure reads spanning two or more exons are not penalized


* Structural Variation Detection
    * Use DNA to determine structural variation of chromosomes
    * Duplications lead to increased read coverage for certain regions compared to control genomes
    * Inversions look like they have flipped reads that map relatively far away compared to control


* Metagenomics
    * Programs take database of all organisms, then compares all DNA in a sample (e.g. of gut bacteria or sea bacteria across the world) to that database to find which organisms are in that sample and in what proportion
    

* Assembly verification
    * Map all your reads back to your assembled genome to make sure your scaffold sequences came from *somewhere*.
    
    
* Medical exome sequencing
    * Take exons and generate complementary sequences for those exons
    * Use RNA bait to pull out those exon sequences, then sequence and compare back to the reference genome to find where there might have been disease-causing mutations
    
    
* BLAST: Optimal Alignment
    * Smith-Waterman alignment, dynamic programming
    1. BLAST provides an optimal alignment using dynamic program
    2. BLAST was designed for searching for distantly related homologues
    3. BLAST is optimized for gene-sized lengths of sequence
    4. Computational complexity: O(m\*n)
    5. When **n** is a reference genome, the problem is not tractable
    
    
* Alignment Algorithms
    * BLAST and BLAT vs. Short-read aligners
    
        1. Must be capable of aligning hundreds of millions of reads
            * Extremely fast -- less sensitive
        2. Requires indexing and storing the reference genome in a special data structure
        3. Can typically be multithreaded to increase efficiency
        
    * Types of alignments
        1. Placement alignments
            * Only the best placement needs to be calculated
            * Reads will be fully aligned by additional software once they are in the correct location
            * Bowtie, BWA
            * Exome sequencing for disease detection
                * Can be hard to distinguish real de novo mutations from sequencing error
        2. Full alignments
            * Place and fully align the read in one step
            * GSnap
            * RNA-seq, RAD-seq alignments
            
    * K-mer hashing algorithm
        * GSnap
        * More flexible, perhaps too promiscuous
        * Can handle intron/exon splicing directly
        * BLAST matches k-mers to reference, then performed non-gapped extension on those k-mer matches
        * BLAT - BLAST-Like Alignment Tool
            * Three major differences from BLAST:
                1. BLAST builds an index of the query sequence and scans linearly through the BLAST database; BLAT indexes the database and scans linearly through the query sequence
                2. BLAST returns each area of homology as a separate alignment; BLAT stitches alignments together.
                3. BLAT has unique code for intron handling "unsplices" mRNA onto the genome
                
    * Burrows-Wheeler data structure
        * used in data compression, e.g. bzip2
        * Michael Burrows and David Wheeler in 1994 DEC
        * Bowtie, BWA.
        * Extremely compact and fast
        * (-) Requires very close matches to the genome
        * (-) to handle exon/intron splicing, must be coupled with an additional program
        * "magicable"?
        
        * Encode: "aardvark"
        * Rotate the text, add a sentinel character (a character that you don't expect to appear in your word)
        * aardvark\$, ardvark\$a, rdvark\$aa... \$aardvark -> makes all possible suffices and prefices of word
        * Sort the text, then extract last column -> "k\$avrraad". Can now be encoded using run-length encoding: "k\$av2r2ad" (compress the text)
        * ???? wat
        * The Last First Mapping Property
            * The letter in the final column is previous to the letter in the first column.
            * The relative, sorted order of letters in the final column is maintained in the first column
            
        * Bowtie - uses BWT
                
    * Suffix Trees
        * Take a sequence, like banana, and do the same transformation banana\$, This time, only keep the suffixes
        * banana\$, anana\$, nana\$ ... a\$, \$
        
    * Suffix array
        * Take suffixes as in suffix trees, then put into array and sort the array
        * Use a binary search to find your sequence (ideally O(n) complexity)
        
        
* Obtain Reference Genomic Data
    1. Obtain a reference genome:
        * Ensembl, http://ensembl.org
        * UCSC, http://genome.ucsc.edu
    2. Convert the reference genome into fast, searchable data structure
    3. If processing RNA-seq, obtain the gene models
    
    * GTF/GFF files (Gene Feature Format)
    * SAM files (Sequence Alignment/Map format)
        * Standardized format for short-read sequence alignments
        * Supported by all major alignment packages.
        * Cosists of a header section and an alignment section
        * Flag section tells us attibutes about the read -> written in binary
        * CIGAR section tells us about mapping to reference
    * BAM (binary version of SAM)